# Import thư viện cần thiết

In [1]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [2]:
import numpy as np
import pandas as pd
import os
from scipy import sparse
from scipy.sparse import csr_matrix, save_npz, load_npz

# Load file json

In [3]:
def load_json_file(path):
    with open(path, 'r', encoding='utf-8') as f:
        raw = f.read().strip()
        try:
            data = json.loads(raw)
            if isinstance(data, dict):
                data = [data]
        except json.JSONDecodeError:
            data = [json.loads(line) for line in raw.splitlines() if line.strip()]
    return pd.DataFrame(data)

PATH = "data/processed_data/processed_data.json"
df = load_json_file(PATH)


In [4]:
df.head()

,url,title,description,content,metadata,url_img,title_clean,desc_clean,content_clean
0,https://dantri.com.vn/xa-hoi/cao-toc-can-tho-c...,Cao tốc Cần Thơ - Cà Mau dài 110km sẽ có 2 trạ...,"(Dân trí) - Theo chủ đầu tư, cao tốc Cần Thơ -...","Chiều 21/8, tại Km53 đoạn qua xã Vị Thuỷ, TP C...","{'cat': 'Xã hội', 'published_date': '21/08/202...",https://cdnphoto.dantri.com.vn/P1a2PG0E8zh4Id3...,cao_tốc thơ - cà_mau 110 km 2 trạm dừng nghỉ,dân_trí - chủ đầu_tư cao_tốc thơ - cà_mau 2 tr...,chiều 21 8 km53 đoạn xã vị thủy tp thơ cao_tốc...
1,https://dantri.com.vn/xa-hoi/hang-tram-cay-tho...,Hàng trăm cây thông lâu năm cạnh cao tốc chết ...,(Dân trí) - Cơ quan chức năng thành phố Huế đa...,"Chiều 21/8, lãnh đạo Chi cục Kiểm lâm thành ph...","{'cat': 'Xã hội', 'published_date': '21/08/202...",https://cdnphoto.dantri.com.vn/F-T4ANcHV9Usi1X...,hàng trăm thông_lâu cạnh cao_tốc chết khô_nghi...,dân_trí - cơ_quan_chức_năng thành_phố huế điều...,chiều 21 8 lãnh_đạo chi_cục kiểm_lâm thành_phố...
2,https://dantri.com.vn/xa-hoi/bo-chinh-tri-cho-...,Bộ Chính trị cho ý kiến về phương án nhân sự c...,(Dân trí) - Tổng Bí thư Tô Lâm chủ trì cuộc là...,"Ngày 21/8, tại Trụ sở Trung ương Đảng, Tổng Bí...","{'cat': 'Xã hội', 'published_date': '21/08/202...",https://cdnphoto.dantri.com.vn/ctGvNG9uAewwWct...,chính_trị ý_kiến phương_án nhân_sự hà_nội tphcm,dân_trí - tổng_bí_thư tô_lâm chủ_trì làm_việc ...,21 8 trụ_sở trung_ương_đảng tổng_bí_thư tô_lâm...
3,https://dantri.com.vn/xa-hoi/dang-bo-co-quan-d...,Đảng bộ cơ quan Đảng tỉnh Tây Ninh thống nhất ...,"(Dân trí) - Sau 2 ngày làm việc khẩn trương, n...","Đại hội diễn ra trong 2 ngày 20-21/8, với sự t...","{'cat': 'Xã hội', 'published_date': '21/08/202...",https://cdnphoto.dantri.com.vn/a07r8hKzPsT1K-g...,đảng_bộ cơ_quan đảng tỉnh tây_ninh thống_nhất ...,dân_trí - 2 làm_việc khẩn_trương nghiêm_túc đạ...,đại_hội diễn 2 20-21 8 tham_dự 250 đại_biểu đạ...
4,https://dantri.com.vn/xa-hoi/bat-giu-hai-quai-...,"Bắt giữ hai ""quái xế"" lao vào đường cấm, tông ...","(Dân trí) - Trung tướng Nguyễn Thanh Tùng, Giá...","Tối 21/8, trao đổi với phóng viên Dân trí , Tr...","{'cat': 'Xã hội', 'published_date': '21/08/202...",https://cdnphoto.dantri.com.vn/7K0gBoMHT0MVXxO...,bắt_giữ hai quái_xế lao đường cấm tông_gục chi...,dân_trí - trung_tướng nguyễn_thanh_tùng giám_đ...,tối 21 8 trao_đổi phóng_viên dân_trí trung_tướ...


In [5]:
meta_df = pd.json_normalize(df['metadata'])
meta_df.head(20)

,cat,published_date,author,subcat
0,Xã hội,21/08/2025 - 20:27,Bảo Kỳ,
1,Xã hội,21/08/2025 - 20:08,Vi Thảo,
2,Xã hội,21/08/2025 - 20:00,Hoài Thu,
3,Xã hội,21/08/2025 - 20:08,An Huy,
4,Xã hội,21/08/2025 - 22:41,Văn Yên,
5,Xã hội,21/08/2025 - 19:11,An Huy,
6,Xã hội,21/08/2025 - 18:02,Xuân Đoàn,
7,Xã hội,21/08/2025 - 17:45,Xuân Đoàn,
8,Xã hội,21/08/2025 - 17:35,Phùng Minh,
9,Xã hội,21/08/2025 - 17:31,Tiến Thành,


In [6]:
col = meta_df['subcat']

cnt_none = (col.apply(lambda x: x is None)).sum()
cnt_nan  = col.isna().sum()
cnt_empty = (col.astype(str).str.strip() == "").sum()   

print("None (is None):", cnt_none)
print("NaN (isna):    ", cnt_nan)
print('Empty "":      ', cnt_empty)

total_blank = ((col.isna()) | (col.astype(str).str.strip() == "")).sum()
print("Total blank:", total_blank)


None (is None): 335
NaN (isna):     335
Empty "":       3753
Total blank: 4088


In [7]:
meta_df['subcat'] = meta_df['subcat'].replace({
    "": np.nan,
    " ": np.nan,
    "nan": np.nan,
    "None": np.nan
})

# Nếu có giá trị None của Python thì pandas.isna() sẽ bắt được
meta_df['subcat'] = meta_df['subcat'].where(~meta_df['subcat'].isna(), np.nan)

# Kiểm tra lại
print(meta_df['subcat'].value_counts(dropna=False))
print("Tổng số NaN:", meta_df['subcat'].isna().sum())


subcat
NaN                  4755
Tin tức               393
Tuyển sinh            161
Quân sự                94
Thị trường             87
                     ... 
Kiến thức chạy bộ       1
Giới trẻ                1
Xe mới                  1
Bảo hiểm                1
Quỹ Hy vọng             1
Name: count, Length: 124, dtype: int64
Tổng số NaN: 4755


In [8]:
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7881 entries, 0 to 7880
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cat             7881 non-null   object
 1   published_date  7881 non-null   object
 2   author          7881 non-null   object
 3   subcat          3126 non-null   object
dtypes: object(4)
memory usage: 246.4+ KB


In [9]:
df_merge = pd.concat([df.drop(columns=['metadata']), meta_df], axis=1)

In [10]:
df_merge.head()

,url,title,description,content,url_img,title_clean,desc_clean,content_clean,cat,published_date,author,subcat
0,https://dantri.com.vn/xa-hoi/cao-toc-can-tho-c...,Cao tốc Cần Thơ - Cà Mau dài 110km sẽ có 2 trạ...,"(Dân trí) - Theo chủ đầu tư, cao tốc Cần Thơ -...","Chiều 21/8, tại Km53 đoạn qua xã Vị Thuỷ, TP C...",https://cdnphoto.dantri.com.vn/P1a2PG0E8zh4Id3...,cao_tốc thơ - cà_mau 110 km 2 trạm dừng nghỉ,dân_trí - chủ đầu_tư cao_tốc thơ - cà_mau 2 tr...,chiều 21 8 km53 đoạn xã vị thủy tp thơ cao_tốc...,Xã hội,21/08/2025 - 20:27,Bảo Kỳ,NaN
1,https://dantri.com.vn/xa-hoi/hang-tram-cay-tho...,Hàng trăm cây thông lâu năm cạnh cao tốc chết ...,(Dân trí) - Cơ quan chức năng thành phố Huế đa...,"Chiều 21/8, lãnh đạo Chi cục Kiểm lâm thành ph...",https://cdnphoto.dantri.com.vn/F-T4ANcHV9Usi1X...,hàng trăm thông_lâu cạnh cao_tốc chết khô_nghi...,dân_trí - cơ_quan_chức_năng thành_phố huế điều...,chiều 21 8 lãnh_đạo chi_cục kiểm_lâm thành_phố...,Xã hội,21/08/2025 - 20:08,Vi Thảo,NaN
2,https://dantri.com.vn/xa-hoi/bo-chinh-tri-cho-...,Bộ Chính trị cho ý kiến về phương án nhân sự c...,(Dân trí) - Tổng Bí thư Tô Lâm chủ trì cuộc là...,"Ngày 21/8, tại Trụ sở Trung ương Đảng, Tổng Bí...",https://cdnphoto.dantri.com.vn/ctGvNG9uAewwWct...,chính_trị ý_kiến phương_án nhân_sự hà_nội tphcm,dân_trí - tổng_bí_thư tô_lâm chủ_trì làm_việc ...,21 8 trụ_sở trung_ương_đảng tổng_bí_thư tô_lâm...,Xã hội,21/08/2025 - 20:00,Hoài Thu,NaN
3,https://dantri.com.vn/xa-hoi/dang-bo-co-quan-d...,Đảng bộ cơ quan Đảng tỉnh Tây Ninh thống nhất ...,"(Dân trí) - Sau 2 ngày làm việc khẩn trương, n...","Đại hội diễn ra trong 2 ngày 20-21/8, với sự t...",https://cdnphoto.dantri.com.vn/a07r8hKzPsT1K-g...,đảng_bộ cơ_quan đảng tỉnh tây_ninh thống_nhất ...,dân_trí - 2 làm_việc khẩn_trương nghiêm_túc đạ...,đại_hội diễn 2 20-21 8 tham_dự 250 đại_biểu đạ...,Xã hội,21/08/2025 - 20:08,An Huy,NaN
4,https://dantri.com.vn/xa-hoi/bat-giu-hai-quai-...,"Bắt giữ hai ""quái xế"" lao vào đường cấm, tông ...","(Dân trí) - Trung tướng Nguyễn Thanh Tùng, Giá...","Tối 21/8, trao đổi với phóng viên Dân trí , Tr...",https://cdnphoto.dantri.com.vn/7K0gBoMHT0MVXxO...,bắt_giữ hai quái_xế lao đường cấm tông_gục chi...,dân_trí - trung_tướng nguyễn_thanh_tùng giám_đ...,tối 21 8 trao_đổi phóng_viên dân_trí trung_tướ...,Xã hội,21/08/2025 - 22:41,Văn Yên,NaN


In [11]:
print(df_merge['cat'].unique())
df_merge['cat'].value_counts()


['Xã hội' 'Kinh doanh' 'Đời sống' 'Sức khỏe' 'Pháp luật' 'Thế giới'
 'Khoa học' 'Thể thao' 'Giải trí' 'Du lịch' 'Giáo dục' 'Chính trị'
 'Thời sự' 'Dân tộc và Tôn giáo' 'Công nghệ' 'Nghị quyết 57'
 'Thông tin và truyền thông' 'Nội dung chuyên đề' 'Ô tô - Xe máy'
 'Net Zero' 'Bất động sản' 'Văn hóa - Giải trí' 'Ảnh' 'Các bệnh' 'Ý kiến'
 'Mẹo tư vấn']


cat
Thể thao                     775
Kinh doanh                   757
Giáo dục                     756
Thế giới                     752
Du lịch                      733
Sức khỏe                     728
Pháp luật                    695
Giải trí                     613
Đời sống                     531
Xã hội                       343
Khoa học                     243
Thời sự                      217
Văn hóa - Giải trí           188
Công nghệ                    186
Chính trị                    180
Các bệnh                     126
Mẹo tư vấn                    17
Net Zero                       9
Bất động sản                   8
Ý kiến                         7
Ô tô - Xe máy                  6
Thông tin và truyền thông      5
Nội dung chuyên đề             2
Nghị quyết 57                  2
Ảnh                            1
Dân tộc và Tôn giáo            1
Name: count, dtype: int64

In [12]:
cat_map = {
    # 1. Thể thao
    "Thể thao": "Thể thao",

    # 2. Kinh tế
    "Kinh doanh": "Kinh tế",
    "Bất động sản": "Kinh tế",

    # 3. Giáo dục
    "Giáo dục": "Giáo dục",

    # 4. Xã hội
    "Xã hội": "Xã hội",
    "Đời sống": "Xã hội",

    # 5. Du lịch
    "Du lịch": "Du lịch",

    # 6. Sức khỏe
    "Sức khỏe": "Sức khỏe",
    "Các bệnh": "Sức khỏe",

    # 7. Pháp luật
    "Pháp luật": "Pháp luật",

    # 8. Văn hóa - Giải trí
    "Giải trí": "Văn hóa - Giải trí",
    "Văn hóa - Giải trí": "Văn hóa - Giải trí",

    # 9. Chính trị
    "Thời sự": "Chính trị",
    "Chính trị": "Chính trị",

    # 10. Khoa học - Công nghệ
    "Khoa học": "Khoa học - Công nghệ",
    "Công nghệ": "Khoa học - Công nghệ",

    # 11. Thế giới
    "Thế giới": "Thế giới",
}

df_merge['cat_mapped'] = df_merge['cat'].map(cat_map)
print(df_merge['cat_mapped'].value_counts())
df_merge = df_merge[df_merge['cat_mapped'].notna()].copy()
print(df_merge['cat_mapped'].isna().sum()) 

cat_mapped
Xã hội                  874
Sức khỏe                854
Văn hóa - Giải trí      801
Thể thao                775
Kinh tế                 765
Giáo dục                756
Thế giới                752
Du lịch                 733
Pháp luật               695
Khoa học - Công nghệ    429
Chính trị               397
Name: count, dtype: int64
0


In [13]:
df_merge['cat_mapped']

0         Xã hội
1         Xã hội
2         Xã hội
3         Xã hội
4         Xã hội
          ...   
7876      Xã hội
7877      Xã hội
7878      Xã hội
7879      Xã hội
7880    Sức khỏe
Name: cat_mapped, Length: 7831, dtype: object

In [14]:
df_merge.head()

,url,title,description,content,url_img,title_clean,desc_clean,content_clean,cat,published_date,author,subcat,cat_mapped
0,https://dantri.com.vn/xa-hoi/cao-toc-can-tho-c...,Cao tốc Cần Thơ - Cà Mau dài 110km sẽ có 2 trạ...,"(Dân trí) - Theo chủ đầu tư, cao tốc Cần Thơ -...","Chiều 21/8, tại Km53 đoạn qua xã Vị Thuỷ, TP C...",https://cdnphoto.dantri.com.vn/P1a2PG0E8zh4Id3...,cao_tốc thơ - cà_mau 110 km 2 trạm dừng nghỉ,dân_trí - chủ đầu_tư cao_tốc thơ - cà_mau 2 tr...,chiều 21 8 km53 đoạn xã vị thủy tp thơ cao_tốc...,Xã hội,21/08/2025 - 20:27,Bảo Kỳ,NaN,Xã hội
1,https://dantri.com.vn/xa-hoi/hang-tram-cay-tho...,Hàng trăm cây thông lâu năm cạnh cao tốc chết ...,(Dân trí) - Cơ quan chức năng thành phố Huế đa...,"Chiều 21/8, lãnh đạo Chi cục Kiểm lâm thành ph...",https://cdnphoto.dantri.com.vn/F-T4ANcHV9Usi1X...,hàng trăm thông_lâu cạnh cao_tốc chết khô_nghi...,dân_trí - cơ_quan_chức_năng thành_phố huế điều...,chiều 21 8 lãnh_đạo chi_cục kiểm_lâm thành_phố...,Xã hội,21/08/2025 - 20:08,Vi Thảo,NaN,Xã hội
2,https://dantri.com.vn/xa-hoi/bo-chinh-tri-cho-...,Bộ Chính trị cho ý kiến về phương án nhân sự c...,(Dân trí) - Tổng Bí thư Tô Lâm chủ trì cuộc là...,"Ngày 21/8, tại Trụ sở Trung ương Đảng, Tổng Bí...",https://cdnphoto.dantri.com.vn/ctGvNG9uAewwWct...,chính_trị ý_kiến phương_án nhân_sự hà_nội tphcm,dân_trí - tổng_bí_thư tô_lâm chủ_trì làm_việc ...,21 8 trụ_sở trung_ương_đảng tổng_bí_thư tô_lâm...,Xã hội,21/08/2025 - 20:00,Hoài Thu,NaN,Xã hội
3,https://dantri.com.vn/xa-hoi/dang-bo-co-quan-d...,Đảng bộ cơ quan Đảng tỉnh Tây Ninh thống nhất ...,"(Dân trí) - Sau 2 ngày làm việc khẩn trương, n...","Đại hội diễn ra trong 2 ngày 20-21/8, với sự t...",https://cdnphoto.dantri.com.vn/a07r8hKzPsT1K-g...,đảng_bộ cơ_quan đảng tỉnh tây_ninh thống_nhất ...,dân_trí - 2 làm_việc khẩn_trương nghiêm_túc đạ...,đại_hội diễn 2 20-21 8 tham_dự 250 đại_biểu đạ...,Xã hội,21/08/2025 - 20:08,An Huy,NaN,Xã hội
4,https://dantri.com.vn/xa-hoi/bat-giu-hai-quai-...,"Bắt giữ hai ""quái xế"" lao vào đường cấm, tông ...","(Dân trí) - Trung tướng Nguyễn Thanh Tùng, Giá...","Tối 21/8, trao đổi với phóng viên Dân trí , Tr...",https://cdnphoto.dantri.com.vn/7K0gBoMHT0MVXxO...,bắt_giữ hai quái_xế lao đường cấm tông_gục chi...,dân_trí - trung_tướng nguyễn_thanh_tùng giám_đ...,tối 21 8 trao_đổi phóng_viên dân_trí trung_tướ...,Xã hội,21/08/2025 - 22:41,Văn Yên,NaN,Xã hội


In [15]:
df_new = df_merge[['title_clean', 'desc_clean', 'content_clean', 'cat_mapped']].copy()
df_new.head()

,title_clean,desc_clean,content_clean,cat_mapped
0,cao_tốc thơ - cà_mau 110 km 2 trạm dừng nghỉ,dân_trí - chủ đầu_tư cao_tốc thơ - cà_mau 2 tr...,chiều 21 8 km53 đoạn xã vị thủy tp thơ cao_tốc...,Xã hội
1,hàng trăm thông_lâu cạnh cao_tốc chết khô_nghi...,dân_trí - cơ_quan_chức_năng thành_phố huế điều...,chiều 21 8 lãnh_đạo chi_cục kiểm_lâm thành_phố...,Xã hội
2,chính_trị ý_kiến phương_án nhân_sự hà_nội tphcm,dân_trí - tổng_bí_thư tô_lâm chủ_trì làm_việc ...,21 8 trụ_sở trung_ương_đảng tổng_bí_thư tô_lâm...,Xã hội
3,đảng_bộ cơ_quan đảng tỉnh tây_ninh thống_nhất ...,dân_trí - 2 làm_việc khẩn_trương nghiêm_túc đạ...,đại_hội diễn 2 20-21 8 tham_dự 250 đại_biểu đạ...,Xã hội
4,bắt_giữ hai quái_xế lao đường cấm tông_gục chi...,dân_trí - trung_tướng nguyễn_thanh_tùng giám_đ...,tối 21 8 trao_đổi phóng_viên dân_trí trung_tướ...,Xã hội


# Xử lý dữ liệu TF-IDF

In [16]:
from sklearn.compose import ColumnTransformer


In [17]:
OUT_DIR = "./vocab_output"
os.makedirs(OUT_DIR, exist_ok=True)
VOC_TEX = os.path.join(OUT_DIR, "vocabulary.tex")
VOC_CSV = os.path.join(OUT_DIR, "vocabulary.csv")  

In [18]:
assert set(['title_clean','desc_clean','content_clean','cat_mapped']).issubset(df_new.columns)

In [19]:
X = df_new[['title_clean','desc_clean','content_clean']]
y = df_new['cat_mapped']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Train size:", X_train.shape[0])
print("Test size: ", X_test.shape[0])


Train size: 6264
Test size:  1567


In [20]:
tfidf_title = TfidfVectorizer(max_features=3000, ngram_range=(1,2), min_df=2, max_df=0.9)
tfidf_desc  = TfidfVectorizer(max_features=3000, ngram_range=(1,2), min_df=2, max_df=0.9)
tfidf_cont  = TfidfVectorizer(max_features=8000, ngram_range=(1,2), min_df=3, max_df=0.9)

ct = ColumnTransformer(
    transformers=[
        ("title",   tfidf_title, "title_clean"),
        ("desc",    tfidf_desc,  "desc_clean"),
        ("content", tfidf_cont,  "content_clean"),
    ],
    remainder="drop",
    verbose=False
)

In [21]:
_ = ct.fit_transform(X_train)
vec_title_fitted = ct.named_transformers_['title']
vec_desc_fitted  = ct.named_transformers_['desc']
vec_cont_fitted  = ct.named_transformers_['content']



In [22]:
def terms_idf_df(vec: TfidfVectorizer, prefix: str) -> pd.DataFrame:
    return pd.DataFrame({
        "feature": [f"{prefix}__{t}" for t in vec.get_feature_names_out()],
        "source": prefix,
        "term":   vec.get_feature_names_out(),
        "idf":    vec.idf_,
    })

# tạo bảng vocab từ CÁC VEC ĐÃ FIT
v_title = terms_idf_df(vec_title_fitted, "title")
v_desc  = terms_idf_df(vec_desc_fitted,  "desc")
v_cont  = terms_idf_df(vec_cont_fitted,  "content")
vocab_df = pd.concat([v_title, v_desc, v_cont], ignore_index=True)

# Sắp theo đúng thứ tự cột của ma trận (nếu bạn cần đồng bộ hoá)
feature_order = ct.get_feature_names_out()
vocab_df = vocab_df.set_index("feature").loc[feature_order].reset_index()

# (tuỳ chọn) sắp theo idf giảm dần
vocab_sorted = vocab_df.sort_values("idf", ascending=False).reset_index(drop=True)

# Lưu CSV (tiện dùng code)
vocab_sorted.to_csv(VOC_CSV, index=False)

# Xuất LaTeX
def latex_escape(s: str) -> str:
    return (s.replace('\\','\\textbackslash ')
             .replace('_','\\_').replace('&','\\&').replace('%','\\%')
             .replace('#','\\#').replace('{','\\{').replace('}','\\}')
             .replace('$','\\$'))

with open(VOC_TEX, "w", encoding="utf-8") as f:
    f.write("\\documentclass{article}\n\\usepackage[utf8]{vietnam}\n\\usepackage{booktabs}\n"
            "\\usepackage{geometry}\n\\geometry{margin=1in}\n\\begin{document}\n"
            "\\section*{Vocabulary TF--IDF}\n"
            "\\noindent Nguồn (title/desc/content), từ/cụm từ và trọng số IDF được fit trên tập train.\\\\\n"
            "\\begin{tabular}{lll}\n\\toprule\nNguồn & Từ/Cụm từ & IDF\\\\\n\\midrule\n")
    LIMIT = 10000
    for _, r in vocab_sorted.head(LIMIT).iterrows():
        f.write(f"{r['source']} & {latex_escape(r['term'])} & {r['idf']:.4f}\\\\\n")
    f.write("\\bottomrule\n\\end{tabular}\n\\end{document}\n")

print(f"Saved LaTeX vocabulary to: {VOC_TEX}")
print(f"Saved vocabulary CSV to: {VOC_CSV}")


Saved LaTeX vocabulary to: ./vocab_output/vocabulary.tex
Saved vocabulary CSV to: ./vocab_output/vocabulary.csv


# Training

In [23]:
Xtr = ct.fit_transform(X_train)
Xte = ct.transform(X_test)

print("Train shape:", Xtr.shape, "| Test shape:", Xte.shape)

Train shape: (6264, 14000) | Test shape: (1567, 14000)


In [24]:
# Đếm số lượng từng category trong tập test
print("Số lượng bài báo theo category trong tập test")
print(y_test.value_counts())

Số lượng bài báo theo category trong tập test
cat_mapped
Xã hội                  175
Sức khỏe                171
Văn hóa - Giải trí      160
Thể thao                155
Kinh tế                 153
Giáo dục                151
Thế giới                151
Du lịch                 147
Pháp luật               139
Khoa học - Công nghệ     86
Chính trị                79
Name: count, dtype: int64


In [26]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

# ===== Linear SVM =====
svm_clf = LinearSVC(class_weight='balanced', random_state=42)
svm_clf.fit(Xtr, y_train)
svm_pred = svm_clf.predict(Xte)

print(classification_report(y_test, svm_pred, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_test, svm_pred))
print("Accuracy:", accuracy_score(y_test, svm_pred))
print("F1-macro:", f1_score(y_test, svm_pred, average='macro'))
print("F1-weighted:", f1_score(y_test, svm_pred, average='weighted'))

                      precision    recall  f1-score   support

           Chính trị     0.6032    0.4810    0.5352        79
             Du lịch     0.8581    0.9048    0.8808       147
            Giáo dục     0.8931    0.9404    0.9161       151
Khoa học - Công nghệ     0.7887    0.6512    0.7134        86
             Kinh tế     0.8253    0.8954    0.8589       153
           Pháp luật     0.7911    0.8993    0.8418       139
            Sức khỏe     0.9101    0.9474    0.9284       171
            Thế giới     0.9645    0.9007    0.9315       151
            Thể thao     0.9870    0.9806    0.9838       155
  Văn hóa - Giải trí     0.8780    0.9000    0.8889       160
              Xã hội     0.6646    0.6000    0.6306       175

            accuracy                         0.8488      1567
           macro avg     0.8331    0.8273    0.8281      1567
        weighted avg     0.8448    0.8488    0.8451      1567

Confusion matrix:
 [[ 38   1   1   0   2  22   0   0   0   1  14]
 

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import numpy as np
import warnings

warnings.filterwarnings("ignore")

# === 1) Cấu hình CV & thước đo ===
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scorer = 'f1_macro'

# === 2) Lưới tham số cho LinearSVC ===
# Gồm 2 "nhánh" cấu hình:
#  - penalty L2 (mặc định) với dual=True (nhanh cho D >> N, hợp dữ liệu text)
#  - penalty L1 với dual=False (feature selection nhẹ), chỉ hợp lệ với loss='squared_hinge'
param_grid = [
    {
        "C": [0.1, 0.3, 1, 3, 10],
        "class_weight": [None, "balanced"],
        "loss": ["squared_hinge"],
        "penalty": ["l2"],
        "dual": [True],                  # nhanh & ổn cho TF-IDF (sparse, chiều cao)
        "tol": [1e-4, 1e-5],
        "max_iter": [2000, 5000],
    },
    {
        "C": [0.1, 0.3, 1, 3],
        "class_weight": [None, "balanced"],
        "loss": ["squared_hinge"],
        "penalty": ["l1"],               # chỉ dùng được khi dual=False
        "dual": [False],
        "tol": [1e-4, 1e-5],
        "max_iter": [3000, 7000],
    },
]

base = LinearSVC(random_state=42)

grid = GridSearchCV(
    estimator=base,
    param_grid=param_grid,
    scoring=scorer,
    n_jobs=-1,
    cv=cv,
    verbose=1
)

# === 3) Fit trên TRAIN ===
grid.fit(Xtr, y_train)

print("Best params:", grid.best_params_)
print("Best CV f1_macro:", grid.best_score_)

best_svm: LinearSVC = grid.best_estimator_

# === 4) Đánh giá trên TEST ===
svm_pred = best_svm.predict(Xte)

print("\n=== Linear SVM (tuned) on TEST ===")
print(classification_report(y_test, svm_pred, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_test, svm_pred))
print("Accuracy:", accuracy_score(y_test, svm_pred))
print("F1-macro:", f1_score(y_test, svm_pred, average='macro'))
print("F1-weighted:", f1_score(y_test, svm_pred, average='weighted'))


Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best params: {'C': 1, 'class_weight': 'balanced', 'dual': False, 'loss': 'squared_hinge', 'max_iter': 3000, 'penalty': 'l1', 'tol': 0.001}
Best CV f1_macro: 0.833405327898683

=== Linear SVM (tuned) on TEST ===
                      precision    recall  f1-score   support

           Chính trị     0.6133    0.5823    0.5974        79
             Du lịch     0.8344    0.8912    0.8618       147
            Giáo dục     0.9085    0.9205    0.9145       151
Khoa học - Công nghệ     0.7424    0.5698    0.6447        86
             Kinh tế     0.8187    0.8562    0.8371       153
           Pháp luật     0.7881    0.8561    0.8207       139
            Sức khỏe     0.9050    0.9474    0.9257       171
            Thế giới     0.9189    0.9007    0.9097       151
            Thể thao     0.9934    0.9742    0.9837       155
  Văn hóa - Giải trí     0.8720    0.8938    0.8827       160
              Xã hội     0.6358    0.5886   

# LTSM

In [29]:
!pip install tensorflow keras

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.3-cp313-cp313-macosx_10_13_universal2.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached MarkupSafe-3.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.7/200.7 MB 8.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 15.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.8/663.8 kB 8.4 MB/s eta 0:00:00
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.3-cp313-cp313-macosx_10_13_universal2.whl (205 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 20.4 MB/s eta 0:00:00a 0:00:01
Using cached urllib3-2.5.0-py3

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Giả sử bạn đã có df_merge với cột 'title_clean','desc_clean','content_clean','cat_mapped'
df_text = (df_merge['title_clean'] + " " + df_merge['desc_clean'] + " " + df_merge['content_clean']).astype(str)
labels = df_merge['cat_mapped'].astype(str)

# Encode nhãn sang số
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(labels)
y_cat = to_categorical(y)  # one-hot cho softmax

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(df_text, y_cat, test_size=0.2, stratify=y, random_state=42)

# Tokenizer
max_words = 20000   # vocab size
max_len = 300       # độ dài chuỗi tối đa
tok = Tokenizer(num_words=max_words, oov_token="<OOV>")
tok.fit_on_texts(X_train)

Xtr_seq = tok.texts_to_sequences(X_train)
Xte_seq = tok.texts_to_sequences(X_test)

Xtr_pad = pad_sequences(Xtr_seq, maxlen=max_len, padding='post')
Xte_pad = pad_sequences(Xte_seq, maxlen=max_len, padding='post')

print("Train shape:", Xtr_pad.shape, y_train.shape)


Train shape: (6264, 300) (6264, 11)


In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

num_classes = y_cat.shape[1]

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [32]:
history = model.fit(
    Xtr_pad, y_train,
    epochs=10,
    batch_size=64,
    validation_split=0.1,
    verbose=1
)

# Evaluate
loss, acc = model.evaluate(Xte_pad, y_test, verbose=0)
print("Test accuracy:", acc)


Epoch 1/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 42s 451ms/step - accuracy: 0.1680 - loss: 2.2887 - val_accuracy: 0.1754 - val_loss: 2.1356
Epoch 2/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 41s 463ms/step - accuracy: 0.3080 - loss: 1.9598 - val_accuracy: 0.3971 - val_loss: 1.7747
Epoch 3/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 42s 469ms/step - accuracy: 0.4909 - loss: 1.5678 - val_accuracy: 0.5407 - val_loss: 1.4295
Epoch 4/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 42s 469ms/step - accuracy: 0.6014 - loss: 1.2659 - val_accuracy: 0.5901 - val_loss: 1.2738
Epoch 5/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 42s 470ms/step - accuracy: 0.6234 - loss: 1.1812 - val_accuracy: 0.5885 - val_loss: 1.2340
Epoch 6/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 42s 474ms/step - accuracy: 0.6961 - loss: 0.9475 - val_accuracy: 0.6156 - val_loss: 1.2280
Epoch 7/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 42s 471ms/step - accuracy: 0.6729 - loss: 1.0243 - val_accuracy: 0.6316 - val_loss: 1.2040
Epoch 8/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 42s 471ms/step - accuracy: 0.7422 - loss: 0.8228 - val_accu